In [11]:
import pandas as pd
import numpy as np

# Load your data
data = pd.read_csv('political_data.csv')

# Preprocess text (you can expand this as needed)
data['cleaned_text'] = data['text'].str.lower().str.replace('[^\w\s]', '')

/var/folders/cy/zk4z3gfd61j8bl6zhmcvx7yw0000gn/T/ipykernel_2225/2041083594.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  data['cleaned_text'] = data['text'].str.lower().str.replace('[^\w\s]', '')


In [12]:
data.head()

,text,subreddit,post_or_comment,expected_political_leaning,score,cleaned_text
0,My one and only response to this event. Don’t ...,Democrats,post,lib,37956,my one and only response to this event dont yo...
1,PSA for those visiting the sub for the first t...,Democrats,comment,lib,1,psa for those visiting the sub for the first t...
2,Don’t forget burning down Planned Parenthood c...,Democrats,comment,lib,663,dont forget burning down planned parenthood cl...
3,Don't forget how they laughed and cheered when...,Democrats,comment,lib,544,dont forget how they laughed and cheered when ...
4,There's zero chance that this is going to make...,Democrats,comment,lib,126,theres zero chance that this is going to make ...


In [13]:
from gensim.models import Word2Vec

# Tokenize text data
sentences = [text.split() for text in data['cleaned_text']]

# Train a Word2Vec model on the sentences
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Generate embeddings for each document
data['embedding'] = data['cleaned_text'].apply(
    lambda x: np.mean([model.wv[word] for word in x.split() if word in model.wv], axis=0) 
    if any(word in model.wv for word in x.split()) 
    else np.zeros(model.vector_size)
)


In [14]:
data['upvote_weight'] = np.log1p(data['score'])
data['weighted_embedding'] = data.apply(lambda row: row['embedding'] * row['upvote_weight'], axis=1)

/Users/nishk/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/nishk/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/cy/zk4z3gfd61j8bl6zhmcvx7yw0000gn/T/ipykernel_2225/1912533601.py:2: RuntimeWarning: invalid value encountered in multiply
  data['weighted_embedding'] = data.apply(lambda row: row['embedding'] * row['upvote_weight'], axis=1)


In [20]:
import faiss
# Convert embeddings to a contiguous numpy array of type float32
embeddings_matrix = np.vstack(data['weighted_embedding'].values).astype('float32')
embeddings_matrix = np.ascontiguousarray(embeddings_matrix)

# Now, create the FAISS index and add the embeddings
d = embeddings_matrix.shape[1]  # dimension of embeddings
index = faiss.IndexFlatL2(d)    # Euclidean distance, or use IndexFlatIP for inner product
index.add(embeddings_matrix)    # Add embeddings to the index


ValueError: input not a numpy array